The necessary functions can be accessed either of two ways:

1. Install the package locally and import from the local install. With this method, you could install the package into your Python or environment scripts and access it in any folder/notebook without having to reinstall or move files after the initial install. For this method, you will need to save the folder [package_builder/](package_builder/) and all its contents locally, then run the following in the command line:

``` 
pip install <path/to/package_builder>/package_builder/.
```

2. Build the below file structure into your repository, then treat categories.py as a local module. If your_notebook.ipynb is not in the same directory as categories.py, some minor changes to the sys module, shown below, will still allow access to the module.

        ├── categories.py
        ├──data
        │   └── raw
        │       └── epa_categories.xml

In [1]:
#Version 1: Import the functions from the fully installed package.
from ncc_categories.cats import queryAll, printTree, listCategories, singleQuery

[14:51:39] SMARTS Parse Error: syntax error while parsing: [#7](=[#8])$[[#1],[#6X4]]{1..;xm}
[14:51:39] SMARTS Parse Error: Failed parsing SMARTS '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}' for input: '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}'
[14:51:39] SMARTS Parse Error: syntax error while parsing: $[[#1],[#6X4]]{1..;xm}[#7+][#8-]
[14:51:39] SMARTS Parse Error: Failed parsing SMARTS '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]' for input: '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]'
[14:51:39] SMARTS Parse Error: syntax error while parsing: c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}
[14:51:39] SMARTS Parse Error: Failed parsing SMARTS 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}' for input: 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}'
[14:51:39] SMARTS Parse Error: syntax error while parsing: $[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]
[14:51:39] SMARTS Parse Error: Failed parsing SMARTS '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]' for input: '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]'
[14:51:39] SMARTS Parse Error: syntax error while parsing: [#8]{>-

In [ ]:
#Version 2: Import functions from local module.
import sys

sys.path.insert(0,'../')

from categories import queryAll, printTree, listCategories, singleQuery

[14:51:44] SMARTS Parse Error: syntax error while parsing: [#7](=[#8])$[[#1],[#6X4]]{1..;xm}
[14:51:44] SMARTS Parse Error: Failed parsing SMARTS '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}' for input: '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}'
[14:51:44] SMARTS Parse Error: syntax error while parsing: $[[#1],[#6X4]]{1..;xm}[#7+][#8-]
[14:51:44] SMARTS Parse Error: Failed parsing SMARTS '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]' for input: '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]'
[14:51:44] SMARTS Parse Error: syntax error while parsing: c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}
[14:51:44] SMARTS Parse Error: Failed parsing SMARTS 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}' for input: 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}'
[14:51:44] SMARTS Parse Error: syntax error while parsing: $[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]
[14:51:44] SMARTS Parse Error: Failed parsing SMARTS '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]' for input: '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]'
[14:51:44] SMARTS Parse Error: syntax error while parsing: [#8]{>-